In [9]:
# First, install required packages
!pip install SpeechRecognition
!pip install pyaudio
!pip install eng-to-ipa
!pip install pandas

In [11]:
import speech_recognition as sr
import eng_to_ipa
import pandas as pd
from datetime import datetime
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

class RealtimePhoneticAnalyzer:
    def __init__(self):
        self.recognizer = sr.Recognizer()
        self.results = []

    def record_audio(self, duration=5):
        """Record audio from microphone for specified duration"""
        with sr.Microphone() as source:
            print(f"\nAdjusting for ambient noise... Please wait...")
            self.recognizer.adjust_for_ambient_noise(source, duration=1)

            print(f"\nRecording for {duration} seconds...\n")
            audio = self.recognizer.listen(source, timeout=duration)
            print("Recording complete!\n")

            return audio

    def get_ipa_pronunciation(self, text):
        """Convert text to IPA phonetic notation"""
        try:
            ipa = eng_to_ipa.convert(text)
            return ipa if ipa != text else "Conversion failed"
        except:
            return "IPA conversion failed"

    def get_phonemes(self, text):
        """Get individual phonemes"""
        try:
            return eng_to_ipa.separate_words(text)
        except:
            return "Phoneme separation failed"

    def analyze_speech(self, duration=5):
        """Record and analyze speech"""
        try:
            # Record audio
            audio = self.record_audio(duration)

            # Convert speech to text
            text = self.recognizer.recognize_google(audio)
            print(f"Recognized Text: {text}\n")

            # Analyze each word
            words = text.split()
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            for word in words:
                analysis = {
                    'Timestamp': timestamp,
                    'Word': word,
                    'IPA': self.get_ipa_pronunciation(word),
                    'Phonemes': self.get_phonemes(word)
                }
                self.results.append(analysis)

            # Display results
            self.display_results()

        except sr.UnknownValueError:
            print("Could not understand the audio")
        except sr.RequestError as e:
            print(f"Error with the speech recognition service; {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

    def display_results(self):
        """Display analysis results in a formatted table"""
        df = pd.DataFrame(self.results)
        display(HTML(df.to_html(index=False)))

    def save_report(self, filename='phonetic_analysis_report.csv'):
        """Save analysis results to a CSV file"""
        df = pd.DataFrame(self.results)
        df.to_csv(filename, index=False)
        print(f"\nReport saved as {filename}")

    def clear_results(self):
        """Clear previous results"""
        self.results = []

def record_and_analyze(duration=5):
    """Function to record and analyze speech"""
    analyzer = RealtimePhoneticAnalyzer()
    analyzer.analyze_speech(duration)
    return analyzer

def multiple_recordings(num_recordings=3, duration=5):
    """Function to make multiple recordings"""
    analyzer = RealtimePhoneticAnalyzer()

    for i in range(num_recordings):
        print(f"\nRecording {i+1} of {num_recordings}")
        analyzer.analyze_speech(duration)

    return analyzer

In [14]:
# For a single 5-second recording:
analyzer = record_and_analyze(duration=5)

# OR for multiple recordings:
# analyzer = multiple_recordings(num_recordings=3, duration=5)

# To save the results:
# analyzer.save_report('my_phonetic_analysis.csv')


Adjusting for ambient noise... Please wait...

Recording for 5 seconds...

Recording complete!

Recognized Text: hello how r u



Timestamp,Word,IPA,Phonemes
2025-02-05 12:21:45,hello,hɛˈloʊ,Phoneme separation failed
2025-02-05 12:21:45,how,haʊ,Phoneme separation failed
2025-02-05 12:21:45,r,ɑr,Phoneme separation failed
2025-02-05 12:21:45,u,ju,Phoneme separation failed
